In [ ]:
import os
import sys
import json
from collections import Counter
from scipy.integrate import solve_ivp
from scipy.stats import ttest_rel
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi, sp2v, dist, \
    traj_speed, min_sep
from packages.ode_simulator import ODESimulator
from packages.models import cohen_avoid_heading
from sklearn.metrics import mean_squared_error
from fitting import Bai_movObst1, Cohen_movObst1, Cohen_movObst2
from parameters import approaches, avoidances

# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

data = None
def load_data(filename):
    global data
    filename = filename + '.pickle'
    file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', filename))
    with open(file, 'rb') as f:
        data = pickle.load(f)

# file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Cohen_movObst1_data_30Hz.pickle'))
# with open(file, 'rb') as f:
#     data = pickle.load(f)


In [ ]:
'''Simulation with var0'''
%matplotlib qt
Hz = 100
xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 5, 5, 0, 0
vxo0, vyo0, vx0, vy0 = -1, 0, 0, 0.9
s0, phi0 = v2sp([vx0, vy0])
a0 = dphi0 = ds0 = 0
w = 0.1
# xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, a, phi, s, dphi, ds = var0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, a0, phi0, s0, dphi0, ds0, w]
fajen_approach['ps'] = 1
fajen_approach2['ps'] = 1
cohen_avoid2['ps'] = 1
models = [fajen_approach2, cohen_avoid3]
dw = [0.1] * 1 * Hz + [0] * 9 * Hz
args = {'w_goal': 0.1, 'w_obst': 0.1, 'dw': 0}
sim = ODESimulator(Hz=Hz, models=models, args=args, ref=[0, 1])
sim.simulate(var0, total_time=10)
sim.play()

In [ ]:
'''Simulate one trial approach'''
############
i_trial = 0
approach = approaches['Bai_movObst1b']['fajen_approach']['differential_evolution'][0]
avoid = None
############
%matplotlib qt
models = [approach]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='stimuli_onset', t_end='stimuli_out', ps='trial')
print(sim.test('p_dist'))
sim.play(obst=False)

### Cohen_movObst1

In [ ]:
'''Simulate one trial avoidance'''
load_data('Cohen_movObst1_data')
############
i_trial = 0
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
# avoid = avoidances['Cohen_movObst1']['cohen_avoid']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid_heading']['differential_evolution'][-1]
# avoid = {'name': 'cohen_avoid_heading', 'b1': 2.1260796879288923, 'k1': 673.0529452234115, 'c5': 14.968106833780706, 'c6': 1.0453773247966653}

############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='obst_onset', t_end='obst_out', ps='trial')
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
print(sim.test('p_dist'))
sim.play(title=title)

In [18]:
'''Simuate all trials model 1'''
load_data('Cohen_movObst1_data')
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid']['differential_evolution']['-1obst_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Cohen_movObst1(subjects)
sim1.reset()
sim1.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim1.simulate_all(trials1, t_start='obst_onset', t_end='obst_out', ps='trial')
errors = sim1.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors)} sd is {np.std(errors)}")
print(f"order accuracy is {sim1.test('order_accuracy')}")
print(f"heading RMSE is {sim1.test('phi_RMSE')}")
print(f"speed RMSE is {sim1.test('s_RMSE')}")
errors = sim1.test('p_dist', all_errors=True)
# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x**2 for x in errors]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim1.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

Loading finished
Simulated 995 trials in 0:00:32
distance error is 0.19077652321401395 sd is 0.16680774873852128
order accuracy is 0.84321608040201
heading RMSE is 0.1836496881408854
speed RMSE is 0.12293783996439665
The BIC of model cohen_avoid is -2676.4835925817297, MSE is 0.06422050684884084, k is 8, n is 995


In [ ]:
'''Cross validation error'''
load_data('Cohen_movObst1_data')
def cross_validate(folder):
    path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', folder))
    filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
    errors = {}
    for filename in filenames:
        with open(filename, 'rb') as f:
            best = 0
            e_min = float('inf')
            if filename[-6] == '_' or filename[-6] == '-':
                subj_id = int(filename[-5])
            elif filename[-6] == 'l':
                subj_id = -1
            else:
                subj_id = int(filename[-6:-4])
            for i, line in enumerate(f):
                if i > 10:
                    try:
                        err = str(line).split("\\t")[2][:10]
                        if err[0] == '0':
                            err = float(err)
                            if err < e_min:
                                e_min = err
                                best = str(line).replace("\\", "")
                    except:
                        pass
        try:
            if 'acceleration' in path:
                models = json.loads(''.join(best.split('t')[1] + best.split('t')[2]).replace("\'", "\"")) # If contains acceleration model
                models[0]['name'] = "acceleration_approach" # If contains acceleration model
            else:
                models = json.loads(best.split('t')[1].replace("\'", "\"")) # If not contains acceleration model
            subjects = [int(subj_id)]
            sim, trials = Cohen_movObst1(subjects)
            sim.reset()
            sim.models = models
            sim.simulate_all(trials, t_start='obst_onset', t_end='obst_out', ps='trial')
            errors[subj_id] = sim.test('p_dist', all_errors=True)
            print(f'{subj_id} finished')
            print(f'filename {filename}')
            print(f'models {sim.models}')
            print(f'error {np.mean(errors[subj_id])}')
        except Exception as e:
            print(e)
    return errors

    
errors = cross_validate('Cohen_movObst1_all_all_models_obst_onset')
# Compute BIC
for key, val in errors.items():
    print(key, np.mean(val))
'''BIC = n*ln(MSE) + k*ln(n))'''
mses = [x for a in errors.values() for x in a]
mses = [x**2 for x in mses]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Output errors for analysis in R'''
with open('Cohen_movObst1_CV_errors_obst_onset.csv', 'a') as f:
    f.write('model,subject,error\n')
for i in ['', '2', '3', '4']:
    model = 'cohen_avoid' + i
    errors = cross_validate('Cohen_movObst1_cross_validation_cohen_avoid' + i)
    with open('Cohen_movObst1_CV_errors_obst_onset.csv', 'a') as f:
        for subj, error in errors.items():
            for err in error:
                log = [model, subj, err]
                log = [str(x) for x in log]
                f.write(','.join(log) + '\n')
            

### Cohen_movObst2

In [20]:
'''Simuate all trials model 1'''
load_data('Cohen_movObst2_data')
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst2']['cohen_avoid3']['differential_evolution']['-1obst_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Cohen_movObst2(subjects)
sim1.reset()
sim1.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim1.simulate_all(trials1, t_start='obst_onset', t_end='obst_out', ps='trial')
errors = sim1.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors)} sd is {np.std(errors)}")
print(f"order accuracy is {sim1.test('order_accuracy')}")
print(f"heading RMSE is {sim1.test('phi_RMSE')}")
print(f"speed RMSE is {sim1.test('s_RMSE')}")
errors = sim1.test('p_dist', all_errors=True)
# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x**2 for x in errors]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim1.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

Loading finished
Simulated 875 trials in 0:01:01
distance error is 0.24335346579933698 sd is 0.186658554357187
order accuracy is 0.7908571428571428
heading RMSE is 0.2373975380803746
speed RMSE is 0.08918360556924733
The BIC of model cohen_avoid3 is -2027.677628473123, MSE is 0.09406232523126401, k is 6, n is 875


### Bai_movObst1

In [ ]:
'''Simulate one trial avoidance'''
############
i_trial = 962
approach = approaches['Bai_movObst1b']['acceleration_approach']['differential_evolution'][-1]
# avoid = avoidances['Cohen_movObst1']['cohen_avoid']['differential_evolution'][-1]
avoid = avoidances['Bai_movObst1']['perpendicular_avoid']['differential_evolution'][-1]
# avoid = {'name': 'cohen_avoid_heading', 'b1': 2.1260796879288923, 'k1': 673.0529452234115, 'c5': 14.968106833780706, 'c6': 1.0453773247966653}

############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='threshold_onset', t_end='obst_out', ps='trial')
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
print(sim.test('p_dist'))
sim.play(title=title)

In [ ]:
'''Simuate all trials model 1'''
############
# subject = 1
approach = approaches['Bai_movObst1b']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Bai_movObst1']['cohen_avoid4']['differential_evolution'][-1]
############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Bai_movObst1(subjects)
sim1.reset()
sim1.models = models
sim1.simulate_all(trials1, t_start='threshold_onset', t_end='obst_out', ps='trial')
print(f'mean distance error is {sim1.test("p_dist")}, sd is {np.std(sim1.test("p_dist", all_errors=True))}')
print(f'order accuracy is {sim1.test("order_accuracy")}')
print(f'phi RMSE is {sim1.test("phi_RMSE")}')
print(f'speed RMSE is {sim1.test("s_RMSE")}')
# BIC
sim = sim1
k = len(models[1]) - 2
n = len(sim.p_pred)
MSE = np.mean(sim.test('p_dist', all_errors=True)**2)
BIC = n * np.log(MSE) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {MSE}, k is {k}, n is {n}')


In [ ]:
models

In [ ]:
'''Cross validation error'''
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', 'Bai_movObst1_cross_validation_cohen_avoid3_with_approach_trained_on_Bai_movObst1b_all_product_onset'))
filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
errors = {}
for filename in filenames:
    with open(filename, 'rb') as f:
        best = 0
        e_min = float('inf')
        if filename[-6] == '_' or filename[-6] == '-':
            subj_id = int(filename[-5])
        else:
            subj_id = int(filename[-6:-4])
        for i, line in enumerate(f):
            if i > 10:
                try:
                    err = str(line).split("\\t")[2][:10]
                    if err[0] == '0':
                        err = float(err)
                        if err < e_min:
                            e_min = err
                            best = str(line).replace("\\", "")
                except:
                    pass
    try:
        if 'acceleration' in path:
            models = json.loads(''.join(best.split('t')[1] + best.split('t')[2]).replace("\'", "\"")) # If contains acceleration model
            models[0]['name'] = "acceleration_approach" # If contains acceleration model
        else:
            models = json.loads(best.split('t')[1].replace("\'", "\"")) # If not contains acceleration model
        subjects = [int(subj_id)]
        sim, trials = Bai_movObst1(subjects)
        sim.reset()
        sim.models = models
        sim.simulate_all(trials, t_start='dtheta*dpsi_onset', t_end='obst_out', ps='trial')
        errors[subj_id] = sim.test('p_dist', all_errors=True)
        print(f'{subj_id} finished')
        print(f'filename {filename}')
        print(f'models {sim.models}')
        print(f'error {np.mean(errors[subj_id])}')
    except Exception as e:
        print(e)

In [ ]:
for key, val in errors.items():
    print(key, np.mean(val))
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x for a in errors.values() for x in a]
mses = [x**2 for x in mses]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Compare two simulations'''
%matplotlib qt
e1 = sim1.test('p_dist', all_errors=True)
e2 = sim2.test('p_dist', all_errors=True)
plt.plot(e1, label=sim1.models[1]['name'])
plt.plot(e2, label=sim2.models[1]['name'])
plt.legend()
plt.xlabel('trial#')
plt.ylabel('distance error (m)')

In [ ]:
'''Animate certain trials'''
sim = sim
i_trial = 0
sim.play(i_trial=i_trial)
sim.test('p_dist', i_trials=[i_trial])

In [ ]:
sim.models

In [ ]:
'''Check pass order accuracy of model1 and model2 when model1 is worse'''
acc1 = 0
acc2 = 0
total = 0
for i in np.where(np.array(e1) > np.array(e2))[0]:
    j = sim1.i_trials[i]
    pred_order1 = sim1.pass_order_pred[i]
    pred_order2 = sim2.pass_order_pred[i]
    true_order = data.info['pass_order'][j]
    acc1 += 1 if pred_order1 == true_order else 0
    acc2 += 1 if pred_order2 == true_order else 0
    total += 1
print(acc1, acc2, acc1/total, acc2/total)

In [ ]:
'''Check the error when model1 is worse'''
ee1 = []
ee2 = []
for e1i, e2i in zip(e1, e2):
    if e1i > e2i:
        ee1.append(e1i)
    elif e1i < e2i:
        ee2.append(e1i)
print(np.mean(ee1), np.mean(ee2))

In [ ]:
'''Check the conditions of large errors'''
cons = []
for i in np.where(np.array(e1) > 0.3)[0]:
    j = sim1.i_trials[i]
    speed = data.info['obst_speed'][j]
    angle = data.info['obst_angle'][j]
    cons.append((angle, speed))
c = Counter(cons)
print(c)

In [ ]:
'''BIC'''
sim = sim1
k = 8
n = len(sim.p_pred)
MSE = np.mean(sim.test('p_dist', all_errors=True)**2)
BIC = n * np.log(MSE) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {MSE}, k is {k}, n is {n}')

In [ ]:
'''Find trials in certain range of error'''
sim = sim1
errors = sim.test('p_dist', all_errors=True)
print(f'smallest error {np.argmin(errors)}')
for i, err in enumerate(errors):
    if 0.189 < err < 0.19:
        print(i)

In [ ]:
'''Play simulations'''
sim = sim1
i_trial = 897
print(sim.test('p_dist', i_trials=[i_trial]))
sim.play(i_trial=i_trial, save=True)


In [ ]:
'''Check simulation length'''
ls = []
for i in range(len(sim.p_pred)):
    ls.append(len(sim.p_pred[i]))
np.mean(ls)/30

In [ ]:
'''Speed and heading errors correct vs incorrect trials'''
from sklearn.metrics import mean_squared_error
sim = sim1
phi_MSEs = []
speed_MSEs = []
trials = []
for i in range(len(sim.v_pred)):
    j = sim.i_trials[i]
#     if sim.pass_order_pred[i] == sim.data.info['pass_order'][j]:
    v_subj = np.gradient(sim.p_subj[i], axis=0) * sim.data.Hz
    v_pred = sim.v_pred[i]
    s0, phi0 = v2sp(v_subj)
    s1, phi1 = v2sp(v_pred)
    speed_MSEs.append(mean_squared_error(s0, s1))
    phi_MSEs.append(mean_squared_error(phi0, phi1))
    trials.append(i)
print(sim.models)
print(f'{len(phi_MSEs)} trials in total')
print(f'RMSE on heading is {np.sqrt(np.mean(phi_MSEs))}')
print(f'RMSE on speed is {np.sqrt(np.mean(speed_MSEs))}')
print(f"Distance is {sim.test('p_dist', i_trials=trials)}")

In [ ]:
sim2.test('p_dist')

In [ ]:
'''Minimum Passing Distance'''
%matplotlib qt
sim = sim2
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title('Signed predicted minimum passing distance (SMPD)')
ax.set_ylabel('SMPD (m)')
ax.set_xlabel('normalized time (%)')
ax.set_ylim((-2, 2))
for j, i in enumerate(sim.i_trials):
    t0 = data.info['obst_onset'][i]
    t1 = data.info['obst_out'][i]
    p0 = sim.p_pred[j]
    p1 = data.info['p_obst'][i][t0:t1]
    v0 = sim.v_pred[j]
    v1 = data.info['v_obst'][i][t0:t1]
    t = np.linspace(0, 100, len(p0))
    smpd = []
    for _p0, _p1, _v0, _v1 in zip(p0, p1, v0, v1):        
        smpd.append(min_sep(_p0, _p1, _v0, _v1)[0])
    ax.plot(t, smpd, 'k', linewidth=0.1, alpha=0.5)

In [ ]:
print(len(p0), len(p1), len(sim.i_trials))

In [ ]:
'''Compute pred_order'''
s = sim
orders = []
for i in range(len(s.p_pred)):
    j = s.i_trials[i]
    true_order = s.data.info['pass_order'][j]
    p0 = s.p_pred[i][-1]
    v0 = s.v_pred[i][-1]
    p1 = s.p_obst[i][-1]
    _beta = beta(p0, p1, v0)
    angle = s.data.info['obst_angle'][j]
    pred_order = np.sign(_beta * -angle)
    orders.append(true_order == pred_order)
print(np.mean(orders))

In [ ]:
sim.test('order_accuracy')

In [ ]:
'''t-test between two models'''
err1 = sim.test('p_dist', all_errors=True)
err2 = sim2.test('p_dist', all_errors=True)
print(ttest_rel(err1, err2))
print(f'sd1 = {np.std(err1)}, sd2 = {np.std(err2)}')
print(f'df = {len(err1) - 1}')

In [ ]:
'''Show one trial from batch simulations'''
######## index in simulated trials only
i = 83
########
i_trial = sim.i_trials[i]
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(i, title=title, save=False)

# When beta and dpsi has the same sign it means pass in front, otherwise it means pass from behind
pass_order = sim.data.info['pass_order'][i]
pred = sim.pass_order_pred[i]
print('pass order ', pass_order, 'prediction ', pred)
print(f"err is {sim.test('p_dist', i_trial=i)}")

In [ ]:
'''plot true vs predicted speed by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0.5, 2)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    s_subj, s_pred = traj_speed(subj, Hz), traj_speed(pred, Hz)
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Speed
    axes[(abs(angle), speed)].plot(t, s_subj, 'r')
    axes[(abs(angle), speed)].plot(t, s_pred, 'b')


In [ ]:
'''plot true vs predicted heading by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0, 1.5)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    h_subj, h_pred = v2sp(np.gradient(subj, axis=0) * Hz)[1], v2sp(np.gradient(pred, axis=0) * Hz)[1]
    if h_subj[0] < 0:
        h_subj += pi
        h_pred += pi
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Heading
    axes[(abs(angle), speed)].plot(t, h_subj, 'r')
    axes[(abs(angle), speed)].plot(t, h_pred, 'b')
    if abs(angle) == 112.5 and speed == 1.2:
        print(i, i_trial)
